<a href="https://colab.research.google.com/github/abernauer/Deep-Learning-with-Python/blob/master/Chapter3_Getting_started_with_neural_networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#Chapter 3 Getting started with neural networks


#3.1 Anatomy of a neural networks 

Training a neural network revolves around the following objects:

* Layers which are combined into a network (or model)
* The input data and corresponding targets
* The loss function, which defines the feedback signal used for learning.
* The optimizer, which determines how learning proceeds.

Add diagram of the components and how they interact.

#3.1.1 Layers the building blocks of deep learning

The fundamental data structure in neural networks is the *layer*, to which you were introduced in chapter 2. A layer is a data-processing module that takes as input one or more tensors and that outputs one or more tensors. Some layers are stateless but more frequently layers have a state: the layer's *weights*, one or several tensors learned with stochastic gradient descent, which together contain the network's *knowledge*. 

Different layers are appropriate for different tensor formats and different types of data processing. For instance, simple vector data, stored in 2D tensors of shape (samples, features), is often processed by *densely connected* layers, also called *fully connected* or *dense* layers. Sequence data, stored in 3D tensors of shape (samples, timesteps, features), is typically processed by *recurrent* layers such as an *LSTM* layer.
Image data, stored in 4D tensor, is usually processed by 2D convolution layers.

A metaphor for layers is thinking of them as LEGO bricks for deep learning. Building deep-learning models in Keras is done by clipping together compatible layers to form useful data-transformation pipelines. The notion of layer *compatibility* here refers specifically to the fact that every layer will only accept input tensor of a certain shape and will return output tensors of a certain shape. Example: 

```
from keras import layers

layer = layers.Dense(32, input_shape=(784,))
```

We're creating a layer that will only accept as input 2D tensors where the first dimension is 784(axis 0, the batch dimension, is unspecified, and thus any value would be accepted.) This layer will return a tensor where the first dimension has been transformed to be 32.

Thus this layer can only be connected to a downstream layer that expects 32-dimensional vectors as its input. When using Keras, you don't have to worry about compatibility, because the layers you add to your models are dynamically built to match the shape of the incoming layer. For instance, suppose you write the following: 

```
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(32, input_shape=(784,)))
model.add(layers.Dense(32))
```

The second layer will automatically infer its input shape as being the the output shape of the layer that came before.

#3.1.2 Models: networks of layers

A deep-learning model is a directed, acyclic graph of layers. The most common instance is a linear stack of layers, mapping a single input to a single output.

Some broader networks include:

* Two-branch networks
* Multihead networks
* Inception blocks

The topology of a network defines a *hypothesis space*. You may remember that in chapter 1, we defined machine learning as "searching for useful representations of some input data, within a predefined space of possibilities, using guidance from a feedback signal". By choosing a network topology, you constrain your *space of possibilities* (hypothesis space) to a specific series of tensor operations, mapping input data to output data. What you'll then be searching for is a good set of values for the weight tensors involved in these tensor operations.

#3.1.3 Loss functions and optimizers: keys to configuring the learning process

Once the network architecture is defined, you still have to choose two more things:

* Loss function (objective function)-- The quantity that will be minimized during training. It represents a measure of success for the task at hand.
* Optimizer--Determines how the network will be updated based on the loss function. It implements a specific variant of stochastic gradient descent(SGD).

Choosing the right objective function for the right problem is extremely important: your network will take any shortcut it can, to minimize the loss; so if the objective doesn't fully correlate with success for the task at hand your network will end up doing things you may not have wanted. 

Guidelines for chosing the correct loss:
* binary crossentropy
  - two-class classification problem
* categorical crossentropy
  - many-class classification problem
* mean squared error
  - regression problems
* Connectionist temporal classification
  - sequence-learning problem

#3.2.2 Developing with Keras: a quick overview

The typical Keras workflow looks like this:

1. Define your training data: input tensors and target tensors.
2. Define a network of layers (or *model*) that maps your inputs to your targets.
3. Configure the learning process by choosing a loss function, an optimizer, and some metrics to monitor.
4. Iterate on your training data by calling the fit() method of your model.

There are two ways to define a model: using the *Sequential* class(only for linear stacks of layers, which is the most common architecture) or the *functional* API.

As a refresher, here's a two-layer model defined using the *Sequential* class (note that we're passing the expected shape of the input data to the first layer):

```
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(32, activation='relu', input_shape=(784,)))
model.add(layers.Dense(10, activation='softmax'))
```

And the same model using the functional API:

```
input_tensor = layers.Input(shape=(784,))
x = layers.Dense(32, activation='relu')(input_tensor)

output_tensor = layers.Dense(10, activation='softmax') (x)
model = models.Model(inputs=input_tensor, outputs=output_tensor)

```

With the functional API, you're maniplating the data tensors that the model proccesses and applying layers to this tensor as if they were functions.

Once your model architectre is defined, it doesn't matter whether you used a *Sequential* model or the functional API.

The learning process is configured in the compilation step, where you specify the optimizer and loss function(s) that the model should use, as well as the metrics to monitor during training.

```
from keras import optimizers

model.compile(optimizer=optimizers.RMSprop(lr=0.001),
loss='mse',
metrics=['accuracy'])
```

Finally, the learning process consists of passing Numpy arrays of input data ( and the corresponding target data) to the model via the fit() method, similar to what you would do in SciKit-Learn.

```
model.fit(input_tensor, target_tensor, batch_size=128, epochs=10)

```

#3.4 Classifying movie reviews: a binary classification example

Two-class classification, or binary classifictaion, may be the most widely applied kind of machine-learning problem. In this example, you'll learn to classify movie reviews as positive or negative, based on the text contents of the reviews.



#3.4.1 The IMDB dataset

You'll work with th IMDB dataset: as set of 50,000 highly polarized reviews from the Internet Movie Database. They're split into 25,000 reviews for training and 25,000 reviews for testing, each set consisting of 50% negative and 50% positive reviews.

Why use separate training and test sets? Because you should never test a machine learning model on the same data you used to train it! Just because a model performs well on its training data doesn't mean it will perform well on data it has never seen; and what you care about is your model's performance on new data.

The following code will load the dataset.

In [1]:
from keras.datasets import imdb

(train_data, train_labels),  (test_data, test_labels) = imdb.load_data(num_words=10000)

Using TensorFlow backend.


17465344/17464789 [==============================] - 0s 0us/step


The argument num_words=10000 means you'll only keep the top 10,000 most frequently occurring words in the training data. Rare words will be discarded. This allows you to work with vector data of manageable size.

The variables train_data and test_data are lists of reviews; each review is a list of word indices(encoding a sequence of words). train_labels and test_labels are lists of 0s and 1s, where 0 stands for *negative* and 1 stands for *positive*.

In [2]:
train_data[0]

[1,
 14,
 22,
 16,
 43,
 530,
 973,
 1622,
 1385,
 65,
 458,
 4468,
 66,
 3941,
 4,
 173,
 36,
 256,
 5,
 25,
 100,
 43,
 838,
 112,
 50,
 670,
 2,
 9,
 35,
 480,
 284,
 5,
 150,
 4,
 172,
 112,
 167,
 2,
 336,
 385,
 39,
 4,
 172,
 4536,
 1111,
 17,
 546,
 38,
 13,
 447,
 4,
 192,
 50,
 16,
 6,
 147,
 2025,
 19,
 14,
 22,
 4,
 1920,
 4613,
 469,
 4,
 22,
 71,
 87,
 12,
 16,
 43,
 530,
 38,
 76,
 15,
 13,
 1247,
 4,
 22,
 17,
 515,
 17,
 12,
 16,
 626,
 18,
 2,
 5,
 62,
 386,
 12,
 8,
 316,
 8,
 106,
 5,
 4,
 2223,
 5244,
 16,
 480,
 66,
 3785,
 33,
 4,
 130,
 12,
 16,
 38,
 619,
 5,
 25,
 124,
 51,
 36,
 135,
 48,
 25,
 1415,
 33,
 6,
 22,
 12,
 215,
 28,
 77,
 52,
 5,
 14,
 407,
 16,
 82,
 2,
 8,
 4,
 107,
 117,
 5952,
 15,
 256,
 4,
 2,
 7,
 3766,
 5,
 723,
 36,
 71,
 43,
 530,
 476,
 26,
 400,
 317,
 46,
 7,
 4,
 2,
 1029,
 13,
 104,
 88,
 4,
 381,
 15,
 297,
 98,
 32,
 2071,
 56,
 26,
 141,
 6,
 194,
 7486,
 18,
 4,
 226,
 22,
 21,
 134,
 476,
 26,
 480,
 5,
 144,
 30,
 5535,
 18,

In [3]:
train_labels[0]

1

In [5]:
max([max(sequence) for sequence in train_data])

9999

In [6]:
word_index = imdb.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
decoded_review = ' '.join(
    [reverse_word_index.get(i - 3, '?') for i in train_data[0]]
)

1646592/1641221 [==============================] - 0s 0us/step


#3.4.2 Preparing the data

You can't feed lists of integers into a neural networks. You have to turn your lists into tensors. There are two ways to do that:

* Pad your lists so that they all have the same length, turn them into an integer tensor of shape (samples, word_indices), and then use as the first layer in your network a layer capable of handling such integer tensors ( the Embedding layer, which we'll cover in detail later in the book).

* One-hot encode your lists to turn them into vectors of 0s and 1s. This would mean, for instance, turning the sequence [3, 5] into a 10,000-dimensional vector that would be all 0s except for indices 3 and 5, which would be 1s. Then you could use as the first layer in your network a *Dense* layer, capable of handling floating-point vector data.

In [11]:
import numpy as np

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1
    return results

x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

What the samples look like now:

In [12]:
x_train[0]

array([0., 1., 1., ..., 0., 0., 0.])

We should also vectorize the lables:

In [13]:
y_train = np.array(train_labels).astype('float32')
y_test = np.array(test_labels).astype('float32')

No we can build our network.

#3.4.3 Building your network